## Tutoriel : interagir avec le système de stockage S3 du SSP Cloud (MinIO)

In [3]:
import os

import pandas as pd
import numpy as np
import s3fs
import zipfile

### Récupérer les données d'un challenge

In [4]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [5]:
# Télécharger les données dans le service
PATH_IN = 'samicarret/diffusion/TreatedRoomSmallSet2.zip'
fs.download(PATH_IN, 'data/HC3.zip')

[None]

In [6]:
# Décompresser les données
with zipfile.ZipFile("data/HC3.zip","r") as zip_file:
    zip_file.extractall("data/")

### Lecture des fichiers

In [7]:
file = np.load('data/TreatedRoomSmallSet/Human1/centroid.npy')

In [8]:
file

array([[ 2425.77063278,  -657.46941402],
       [ 1744.17081831,  -944.2440339 ],
       [ 2320.19919293, -2464.72205234]])

In [10]:
audio = np.load('data/TreatedRoomSmallSet/Human1/audio.npy')
adjusted_audio = np.load('data/TreatedRoomSmallSet/Human1/adjusted_audio.npy')

deconvolved = np.load('data/TreatedRoomSmallSet/Human1/deconvolved.npy')
directlines = np.load('data/TreatedRoomSmallSet/Human1/directlines.npy')


In [11]:
rms_audio = np.sqrt(np.mean(np.square(audio), axis=2))  # Calculer RMS sur l'axe des échantillons

# Vérifier la forme des niveaux RMS
print("Shape des niveaux RMS:", rms_audio.shape)  # [N_datapoints, N_Microphones]

# Exemple : Niveaux RMS pour le premier enregistrement
print("Niveaux RMS pour le premier enregistrement:")
print(rms_audio[0])  # Afficher les niveaux RMS du premier enregistrement


Shape des niveaux RMS: (3, 10)
Niveaux RMS pour le premier enregistrement:
[29.70875137 29.20671747 25.63379974 23.46920801 24.2696896  27.74678054
 20.81851468 25.1604534  24.41105036 31.64709987]


In [13]:
# Concaténer les niveaux RMS avec les positions de centroid
data_with_positions = np.concatenate((rms_audio, file), axis=1)

# Vérifier la forme du tableau combiné
print("Shape du tableau combiné:", data_with_positions.shape)  # [N_datapoints, N_Microphones + 2]

# Vérifier les premières lignes du tableau
print("Premières lignes du tableau combiné:")
print(data_with_positions[:5])  # Afficher les premières lignes pour vérification


Shape du tableau combiné: (3, 12)
Premières lignes du tableau combiné:
[[   29.70875137    29.20671747    25.63379974    23.46920801
     24.2696896     27.74678054    20.81851468    25.1604534
     24.41105036    31.64709987  2425.77063278  -657.46941402]
 [   31.34640352    29.93810895    30.00472626    24.26575375
     31.04475679    29.63319339    22.08850504    27.39462351
     26.56226851    38.20548557  1744.17081831  -944.2440339 ]
 [   34.85986726    29.18252056    25.56728984    24.03735916
     23.73848728    27.03857834    22.23435472    25.80769672
     23.98810621    32.3542175   2320.19919293 -2464.72205234]]


In [14]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Charger les données nécessaires (audio.npy et centroid.npy)
audio = np.load('data/TreatedRoomSmallSet/Human1/audio.npy')
centroid = np.load('data/TreatedRoomSmallSet/Human1/centroid.npy')

# Calculer les niveaux RMS pour chaque enregistrement pour les 4 micros
rms_audio = np.sqrt(np.mean(np.square(audio), axis=2))

# Concaténer les niveaux RMS avec les positions de centroid
data_with_positions = np.concatenate((rms_audio, centroid), axis=1)

# Vérifier la forme du tableau combiné
print("Shape du tableau combiné:", data_with_positions.shape)  # [N_datapoints, N_Microphones + 2]

# Vérifier les premières lignes du tableau
print("Premières lignes du tableau combiné:")
print(data_with_positions[:5])  # Afficher les premières lignes pour vérification


Shape du tableau combiné: (3, 12)
Premières lignes du tableau combiné:
[[   29.70875137    29.20671747    25.63379974    23.46920801
     24.2696896     27.74678054    20.81851468    25.1604534
     24.41105036    31.64709987  2425.77063278  -657.46941402]
 [   31.34640352    29.93810895    30.00472626    24.26575375
     31.04475679    29.63319339    22.08850504    27.39462351
     26.56226851    38.20548557  1744.17081831  -944.2440339 ]
 [   34.85986726    29.18252056    25.56728984    24.03735916
     23.73848728    27.03857834    22.23435472    25.80769672
     23.98810621    32.3542175   2320.19919293 -2464.72205234]]


In [19]:
import os

# Créer le dossier data s'il n'existe pas
if not os.path.exists('data'):
    os.makedirs('data')

In [20]:
import numpy as np

# Charger les données nécessaires (audio.npy et centroid.npy)
audio = np.load('data/TreatedRoomSmallSet/Human1/audio.npy')
centroid = np.load('data/TreatedRoomSmallSet/Human1/centroid.npy')

# Calculer les niveaux RMS pour chaque enregistrement pour les 4 micros
rms_audio = np.sqrt(np.mean(np.square(audio), axis=2))

# Concaténer les niveaux RMS avec les positions de centroid
data_with_positions = np.concatenate((rms_audio, centroid), axis=1)

# Sauvegarder les données combinées dans le dossier "data"
np.save('data/donnees_combinees.npy', data_with_positions)


In [21]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Charger les données combinées (niveaux RMS et positions) depuis le dossier "data"
data_with_positions = np.load('data/donnees_combinees.npy')

# Séparer les features (niveaux RMS) et les labels (positions)
X = data_with_positions[:, :4]  # Les 4 premières colonnes sont les features (niveaux RMS)
y = data_with_positions[:, 4:]  # Les colonnes suivantes sont les labels (positions)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle de régression linéaire
model = LinearRegression()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Prédire les positions sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer l'erreur moyenne quadratique (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)

# Exemple de prédiction pour une nouvelle donnée (pour illustrer)
nouvelle_donnee = np.array([[30.0, 28.0, 26.0, 24.0]])  # Nouveaux niveaux RMS à prédire
prediction = model.predict(nouvelle_donnee)
print("Prédiction de la position pour les niveaux RMS :", prediction)

# Sauvegarder le modèle entraîné si nécessaire
# from joblib import dump
# dump(model, 'modele_de_regression.joblib')


Root Mean Squared Error (RMSE): 225.58790131150187
Prédiction de la position pour les niveaux RMS : [[  30.39965014   32.66303892   19.03932157   27.20772662   26.55977958
    38.11476921 1970.49231441 -980.28317514]]
